In [1]:
import os
import pickle as pkl
import numpy as np
import pandas as pd
import joblib
import __init__

Navigated to package root: /home/cyprien/CrystaLLMv2_PKV
Added package root to Python path


In [ ]:
df1 = pd.read_csv('_utils/_evaluation_conditional/dft_results/20250814_060706/TiO2_highBG_candidate_CIFs.csv')
df2 = pd.read_csv('_utils/_evaluation_conditional/dft_results/20250814_070605/TiO2_lowBG_candidate_CIFs.csv')
df3 = pd.read_csv('_utils/_evaluation_conditional/dft_results/20250813_203206/SiO2_lowDen_candidate_CIFs.csv')
df4 = pd.read_csv('_utils/_evaluation_conditional/dft_results/20250813_104046/test.csv')
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

df.to_csv('_utils/_evaluation_conditional/dft_results/combined_candidates.csv', index=False)

In [4]:
!python _utils/_evaluation_conditional/dft_ehull.py \
    --input_csv '_utils/_evaluation_conditional/dft_results/combined_candidates.csv' \
    --output_parquet '_utils/_evaluation_conditional/dft_results/processed/combined_candidates.parquet' \
    --huggingface_dataset 'c-bone/mattergen_den_ehull' \
    --num_workers 64 \
    --load_processed_data 'HF-databases/mattergen_dev/mgen_den_ehull_proc.parquet' \
    --output_cif_dir '_utils/_evaluation_conditional/dft_results/processed/stable_candidates/' \
    --density

Reading input CSV: _utils/_evaluation_conditional/dft_results/combined_candidates.csv
Connecting to Materials Project and preparing phase diagrams per chemical system…
Computing E_hull and formation energy/atom via MP…
100%|███████████████████████████████████████████| 59/59 [00:05<00:00, 10.23it/s]

===== Loading & processing Training Dataset =====
Loading training dataset from Hugging Face: c-bone/mattergen_den_ehull (train split)
Processing training dataset and storing CIFs by composition...
Found existing processed training dataset at HF-databases/mattergen_dev/mgen_den_ehull_proc.parquet.
CIFs processed. Will now build training comps...
Building training comps: 100%|███████| 591964/591964 [00:24<00:00, 24008.43it/s]
Parsing reference CIFs into Structures... (parallel)
Parsing reference structures: 100%|████| 491308/491308 [10:56<00:00, 748.52it/s]

===== Novelty Metrics =====

Checking novelty vs. training dataset...
Novelty pass: 100%|█████████████████████████████| 59/59 [00:01<00

In [3]:
!python _utils/_metrics/dft_ehull.py \
    --input_csv '_utils/_evaluation_files/dft_results/combined_candidates.csv' \
    --output_parquet 'test.parquet' \
    --huggingface_dataset 'c-bone/mattergen_den_ehull' \
    --num_workers 64 \
    --load_processed_data 'HF-databases/mattergen_dev/mgen_den_ehull_proc.parquet' \
    --density

MP data file already exists: mp_computed_structure_entries.json.gz
Setting up MP data provider...
Loading MP entries...
Found 154718 MP entries
Organizing MP entries by chemical system...
Processing MP entries: 100%|██████████| 154718/154718 [00:19<00:00, 7851.80it/s]
Organized 154718 entries across 49906 chemical systems
Reading input CSV: _utils/_evaluation_files/dft_results/combined_candidates.csv
Processing structures and computing E_hull and formation energy per atom...
100%|███████████████████████████████████████████| 59/59 [00:00<00:00, 98.12it/s]

Computing Novelty vs Training Dataset
Loading training dataset from Hugging Face: c-bone/mattergen_den_ehull (train split)
Extracting unique reduced formulas from generated structures...
Extracting reduced formulas: 100%|███████████| 59/59 [00:00<00:00, 43829.96it/s]
Found 3 unique reduced formulas in generated set

===== Loading Training Dataset (for novelty metrics)=====
Loading training dataset from Hugging Face: c-bone/mattergen_d

In [10]:
df = pd.read_parquet('_utils/_evaluation_conditional/dft_results/processed/combined_candidates.parquet')
# save to csv
# df.to_csv('_utils/_evaluation_conditional/dft_results/processed/combined_candidates.csv', index=False)
# filter out if dft_ehull is above 0.1
# df = df[df['dft_ehull'] <= 0.1]
# keep only if the 'Strcture' column has 'highBG' in it
df = df[df['Structure'].str.contains('lowDen')]
# take mean of the BandGap_eV column
# df['BandGap_eV'] = df['BandGap_eV'].astype(float)
# print(np.mean(df['BandGap_eV']))
# print(np.std(df['BandGap_eV']))
# df['Density (g/cm^3)'] = df['Density (g/cm^3)'].astype(float)
# df.reset_index(drop=True, inplace=True)
# df = df.sort_values(by='dft_ehull', ascending=True)
# df
print(np.mean(df['Density (g/cm^3)']))
print(np.std(df['Density (g/cm^3)']))
# print standard error
print(np.std(df['Density (g/cm^3)'])/np.sqrt(len(df)))

1.8264176380581354
0.6157059405215568
0.15897459025339758


In [2]:
df = pd.read_parquet('HF-databases/mattergen_dev/mgen_den_ehull_proc.parquet')
df

,Database,Reduced Formula,CIF,Density (g/cm^3),energy_above_hull,norm_Density (g/cm^3),norm_energy_above_hull
0,Mattergen,KCuAgSb2,data_K2Cu2Ag2Sb4\n_symmetry_space_group_name_H...,5.875,0.05,0.2304,0.5
1,Mattergen,Ba3(CaPb)4,data_Ba6Ca8Pb8\n_symmetry_space_group_name_H-M...,5.517,0.011,0.2164,0.11
2,Mattergen,Sr(LaS2)2,data_Sr4La8S16\n_symmetry_space_group_name_H-M...,4.793,0.0,0.188,0.0
3,Mattergen,CeRuC2,data_Ce4Ru4C8\n_symmetry_space_group_name_H-M ...,7.729,0.012,0.3032,0.12
4,Mattergen,Nd2Al3Hg,data_Nd16Al24Hg8\n_symmetry_space_group_name_H...,6.607,0.025,0.2592,0.25
...,...,...,...,...,...,...,...
591959,Mattergen,PrSc2(TlSe2)3,data_Pr2Sc4Tl6Se12\n_symmetry_space_group_name...,6.425,0.014,0.252,0.14
591960,Mattergen,Ba2AlGa,data_Ba6Al3Ga3\n_symmetry_space_group_name_H-M...,3.791,0.0,0.1487,0.0
591961,Mattergen,LiLa5In4,data_Li1La5In4\n_symmetry_space_group_name_H-M...,6.233,0.035,0.2445,0.35
591962,Mattergen,Rb3LiCe3S8,data_Rb9Li3Ce9S24\n_symmetry_space_group_name_...,3.455,0.031,0.1355,0.31


In [7]:
# Find only rows where the 'Reduced Formula' columns is exactly 'SiO2'
df = df[df['Reduced Formula'] == 'SiO2']
# turn to float
df['Density (g/cm^3)'] = df['Density (g/cm^3)'].astype(float)
# sort by density
# sort by ones with density under 1.95
df = df[df['Density (g/cm^3)'] < 1.95]

# save all the cif files to a folder
import os
output_folder = 'lowden_SiO2_cifs_trainset'
os.makedirs(output_folder, exist_ok=True)
for i, row in df.iterrows():
    cif_str = row['CIF']
    formula = row['Reduced Formula']
    density = row['Density (g/cm^3)']
    # replace all spaces in formula by underscores
    formula = formula.replace(' ', '_')
    filename = f"{i}_{formula}_den{density:.2f}.cif"
    filepath = os.path.join(output_folder, filename)
    with open(filepath, 'w') as f:
        f.write(cif_str)

In [12]:
from __future__ import annotations
import base64, io, webbrowser, threading
import dash
from dash import html, dcc, dash_table, Input, Output, State
from pymatgen.core import Structure
import crystal_toolkit.components as ctc
from crystal_toolkit.settings import SETTINGS

app = dash.Dash(assets_folder=SETTINGS.ASSETS_PATH)
server = app.server

base_struct = Structure.from_str(df.iloc[1]['opt_cif'], fmt='cif')

def make_super(s: Structure, sc=(1,1,2)) -> Structure:
    s2 = s.copy()
    s2.make_supercell(sc)
    return s2

viewer = ctc.StructureMoleculeComponent(make_super(base_struct), id="viewer")

props = lambda s: [
    {"Property":"formula","Value":s.composition.reduced_formula},
    {"Property":"sites","Value":len(s)},
    {"Property":"a,b,c (Å)","Value":", ".join(f"{x:.3f}" for x in s.lattice.abc)},
    {"Property":"α,β,γ (°)","Value":", ".join(f"{x:.1f}" for x in s.lattice.angles)},
]

table = dash_table.DataTable(
    id="props",
    columns=[{"name":"Property","id":"Property"},{"name":"Value","id":"Value"}],
    data=props(make_super(base_struct)),
    style_table={"maxHeight":"80vh","overflowY":"auto"},
    style_cell={"padding":"6px","fontFamily":"ui-monospace","fontSize":14}
)

controls = html.Div([
    html.H2("Crystal viewer", style={"margin":"0 0 8px 0"}),
    dcc.Slider(1, 4, 1, value=2, id="sx", tooltip={"placement":"bottom"}),
    dcc.Slider(1, 4, 1, value=2, id="sy", tooltip={"placement":"bottom"}),
    dcc.Slider(1, 4, 1, value=2, id="sz", tooltip={"placement":"bottom"}),
    dcc.Upload(id="upload", children=html.Div(["Drop CIF or ", html.B("click")]),
               multiple=False, style={"border":"1px dashed #888","padding":"8px","borderRadius":"10px","marginTop":"8px"}),
    html.Div(id="status", style={"fontSize":12,"color":"#666","marginTop":"6px"})
], style={"marginBottom":"8px"})

layout = html.Div([
    controls,
    html.Div([
        html.Div(viewer.layout(), style={"flex":"1","minWidth":0,"height":"80vh"}),
        html.Div(table, style={"width":"380px","minWidth":"280px","marginLeft":"12px"})
    ], style={"display":"flex","gap":"12px"})
], style={"padding":"12px"})

ctc.register_crystal_toolkit(app=app, layout=layout)

@app.callback(
    Output("viewer","data"), Output("props","data"), Output("status","children"),
    Input("sx","value"), Input("sy","value"), Input("sz","value"),
    Input("upload","contents"), State("upload","filename"),
    prevent_initial_call=True
)
def update_view(sx, sy, sz, contents, fname):
    s = base_struct
    msg = f"Supercell = {sx}×{sy}×{sz}"
    if contents and fname and fname.lower().endswith(".cif"):
        b64 = contents.split(",",1)[1]
        s = Structure.from_str(base64.b64decode(b64).decode("utf-8"), fmt="cif")
        msg = f"Loaded {fname}; " + msg
    sc = (int(sx or 1), int(sy or 1), int(sz or 1))
    s_sc = make_super(s, sc)
    return s_sc, props(s_sc), msg

if __name__ == "__main__":
    url = "http://127.0.0.1:8050"
    threading.Timer(0.8, lambda: webbrowser.open(url)).start()
    app.run(debug=True, host="127.0.0.1", port=8050)